# Dual-Transformer Energy Trading System - Training Notebook

This notebook trains two specialized transformer models:
1. **Consumption Transformer**: Predicts household energy consumption
2. **Trading Transformer**: Makes optimal buy/sell/hold trading decisions

## Architecture
- Consumption Transformer: 384d, 6 heads, 5 layers → predicts next 24h consumption
- Trading Transformer: 512d, 8 heads, 6 layers → predicts prices & trading decisions

## Success Criteria
- Consumption MAPE < 15%
- Price MAE < $0.05/kWh
- Cost savings: 20-40% vs baseline
- Battery constraints respected (20-90% SoC)

---

## Section 1: Setup & Environment

Install dependencies, mount Google Drive, verify GPU, set random seeds.

In [ ]:
print("="*70)
print("SECTION 1: SETUP & ENVIRONMENT")
print("="*70)

# Mount Google Drive
print("\n1. Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("   ✓ Google Drive mounted")

In [ ]:
# Clone repository from GitHub
print("\n2. Cloning repository from GitHub...")
import os

# Check if repo already exists
if os.path.exists('/content/subatomic-trading-bot'):
    print("   Repository already exists, pulling latest changes...")
    !cd /content/subatomic-trading-bot && git pull
else:
    print("   Cloning repository...")
    !git clone https://github.com/Jai-Dhiman/subatomic-trading-bot.git /content/subatomic-trading-bot

# Navigate to project directory
os.chdir('/content/subatomic-trading-bot')
print(f"   ✓ Working directory: {os.getcwd()}")

# List directory to verify
print("   ✓ Repository contents:")
!ls -la

In [ ]:
# Copy .env file from Google Drive
print("\n3. Setting up environment variables...")
import shutil

# Copy .env from Google Drive to project directory
env_source = '/content/drive/MyDrive/energymvp/.env'
env_dest = '/content/subatomic-trading-bot/.env'

if os.path.exists(env_source):
    shutil.copy(env_source, env_dest)
    print(f"   ✓ Copied .env from Google Drive")
else:
    raise FileNotFoundError(
        f"❌ .env file not found at {env_source}. "
        "Please create it in your Google Drive with SUPABASE_URL and SUPABASE_KEY"
    )

# Verify .env exists
print("   ✓ Environment file ready")

In [ ]:
# Install dependencies using uv (user preference)
print("\n4. Installing dependencies...")
!pip install -q uv

# Install PyTorch with CUDA support
!uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install other dependencies
!uv pip install pandas numpy scikit-learn matplotlib seaborn supabase python-dotenv tqdm
print("   ✓ Dependencies installed")

In [ ]:
# Set random seeds for reproducibility
print("\n5. Setting random seeds...")
import random
import numpy as np
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Deterministic operations (slight performance cost)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"   ✓ Random seed set to {SEED}")

In [ ]:
# Check GPU availability
print("\n6. Checking GPU availability...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"   Device: {device}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory allocated: {torch.cuda.memory_allocated(0) / 1024**2:.1f} MB")
    print(f"   Memory cached: {torch.cuda.memory_reserved(0) / 1024**2:.1f} MB")
else:
    print("   ⚠️ WARNING: GPU not available, using CPU (training will be slow!)")

In [ ]:
# Import project modules
print("\n7. Importing project modules...")
import sys
sys.path.append('/content/subatomic-trading-bot')

# Data loading
from src.data_integration.data_adapter import (
    load_consumption_data,
    load_pricing_data,
    generate_battery_data,
    merge_all_data
)

# Feature engineering
from src.models.feature_engineering_consumption import ConsumptionFeatureEngineer
from src.models.feature_engineering_trading import TradingFeatureEngineer

# Models
from src.models.consumption_transformer import (
    ConsumptionTransformer,
    ConsumptionLoss,
    calculate_mape
)
from src.models.trading_transformer import (
    TradingTransformer,
    TradingLoss
)

# Training utilities
from src.training.training_utils import (
    create_data_loaders,
    train_epoch,
    validate,
    save_checkpoint,
    load_checkpoint
)

# Trading optimizer
from src.models.trading_optimizer import calculate_optimal_trading_decisions

print("   ✓ All modules imported successfully")

In [ ]:
# Setup logging
print("\n8. Setting up logging...")
import logging
from datetime import datetime

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Create checkpoints directory in Google Drive
os.makedirs('/content/drive/MyDrive/energymvp/checkpoints', exist_ok=True)
print("   ✓ Checkpoints directory: /content/drive/MyDrive/energymvp/checkpoints")
print("   ✓ Logging configured")

print("\n" + "="*70)
print("✅ SETUP COMPLETE")
print("="*70)

## Section 2: Data Loading from Supabase

Load real consumption data, pricing data, and battery trading labels from Supabase.

In [ ]:
print("="*70)
print("SECTION 2: DATA LOADING FROM SUPABASE")
print("="*70)

# Load environment variables
print("\n1. Loading environment variables...")
from dotenv import load_dotenv
load_dotenv()

# Verify Supabase credentials
supabase_url = os.getenv('SUPABASE_URL')
supabase_key = os.getenv('SUPABASE_KEY')

if not supabase_url or not supabase_key:
    raise ValueError(
        "❌ SUPABASE_URL and SUPABASE_KEY must be set in .env file. "
        "Please copy from your local .env to Colab."
    )

print(f"   ✓ Supabase URL: {supabase_url[:30]}...")
print("   ✓ Supabase key configured")

In [ ]:
# Load consumption data from Supabase
print("\n2. Loading consumption data from Supabase...")
print("   (This will load from august11homeconsumption table)")

consumption_df = load_consumption_data(source='supabase')

print(f"   ✓ Loaded {len(consumption_df):,} consumption records")
print(f"   ✓ Date range: {consumption_df['timestamp'].min()} to {consumption_df['timestamp'].max()}")
print(f"   ✓ Houses: {sorted(consumption_df['house_id'].unique())}")
print(f"   ✓ Columns: {consumption_df.columns.tolist()}")

In [ ]:
# Load pricing data from Supabase
print("\n3. Loading pricing data from Supabase...")
print("   (This will load from cabuyingpricehistoryseptember2025 table with LMP filter)")

pricing_df = load_pricing_data()

print(f"   ✓ Loaded {len(pricing_df):,} pricing records")
print(f"   ✓ Price range: ${pricing_df['price_per_kwh'].min():.4f} to ${pricing_df['price_per_kwh'].max():.4f} per kWh")
print(f"   ✓ Mean price: ${pricing_df['price_per_kwh'].mean():.4f} per kWh")
print(f"   ✓ Median price: ${pricing_df['price_per_kwh'].median():.4f} per kWh")

In [ ]:
# Generate battery data
print("\n4. Generating synthetic battery data...")
print("   (This creates battery state data aligned with consumption timestamps)")

battery_df = generate_battery_data(
    timestamps=consumption_df['timestamp'],
    consumption_data=consumption_df
)

print(f"   ✓ Generated {len(battery_df):,} battery state records")
print(f"   ✓ Battery SoC range: {battery_df['battery_soc_percent'].min():.1f}% to {battery_df['battery_soc_percent'].max():.1f}%")

In [ ]:
# Merge all data sources
print("\n5. Merging all data sources...")
print("   (Aligning consumption, pricing, and battery data by timestamp)")

df_complete = merge_all_data(consumption_df, pricing_df, battery_df)

print(f"   ✓ Complete dataset: {len(df_complete):,} records")
print(f"   ✓ Total columns: {len(df_complete.columns)}")
print(f"   ✓ Missing values: {df_complete.isnull().sum().sum()}")

In [ ]:
# Display sample data
print("\n6. Sample data:")
print(df_complete.head())

print("\n7. Data statistics:")
print(df_complete.describe())

In [ ]:
# Validate data quality
print("\n8. Validating data quality...")

# Check for NaN/Inf values
assert not df_complete.isnull().any().any(), "❌ Found NaN values in dataset!"
assert not np.isinf(df_complete.select_dtypes(include=[np.number])).any().any(), "❌ Found Inf values in dataset!"

# Check for required columns
required_cols = [
    'timestamp', 'house_id', 'total_consumption_kwh',
    'price_per_kwh', 'battery_soc_percent'
]
for col in required_cols:
    assert col in df_complete.columns, f"❌ Missing required column: {col}"

print("   ✓ No NaN/Inf values detected")
print("   ✓ All required columns present")

print("\n" + "="*70)
print("✅ DATA LOADING COMPLETE")
print(f"Ready for training with {len(df_complete):,} samples")
print("="*70)

## Section 3: Preflight Validation (CPU)

Test models and data pipeline on CPU before expensive GPU training.

In [ ]:
print("="*70)
print("SECTION 3: PREFLIGHT VALIDATION - CPU ONLY")
print("="*70)
print("\nThis section validates everything works BEFORE GPU training.")
print("We'll run quick tests on CPU to catch errors early.\n")

In [ ]:
# Test 1: Consumption Feature Engineering
print("\n1. Testing Consumption Feature Engineering...")

engineer_consumption = ConsumptionFeatureEngineer()
features_consumption = engineer_consumption.prepare_features(df_complete, fit=True)

print(f"   ✓ Features extracted: {features_consumption.shape}")
print(f"   ✓ Expected: (n_samples, 17)")
assert features_consumption.shape[1] == 17, f"❌ Expected 17 features, got {features_consumption.shape[1]}"

# Check for NaN/Inf
assert not np.isnan(features_consumption).any(), "❌ NaN values in consumption features!"
assert not np.isinf(features_consumption).any(), "❌ Inf values in consumption features!"
print("   ✓ No NaN/Inf values in features")

In [ ]:
# Test 2: Create sequences for Consumption Transformer
print("\n2. Creating sequences for Consumption Transformer...")

# Create sequences using feature engineer
X_cons, y_cons = engineer_consumption.create_sequences(
    features_consumption,
    df_complete['total_consumption_kwh'].values,
    sequence_length=48,
    horizons={'day': 48, 'week': 336}
)

print(f"   ✓ X_cons shape: {X_cons.shape}")
print(f"   ✓ Expected: (n_sequences, 48, 17)")
for key, value in y_cons.items():
    print(f"   ✓ y_cons['{key}'] shape: {value.shape}")

In [ ]:
# Test 3: Consumption Transformer forward pass
print("\n3. Testing Consumption Transformer forward pass (CPU)...")

model_consumption_test = ConsumptionTransformer(
    n_features=17,
    d_model=384,
    n_heads=6,
    n_layers=5,
    horizons={'day': 48, 'week': 336}
)

# Test with small batch
x_test = torch.FloatTensor(X_cons[:2])  # 2 samples
with torch.no_grad():
    output_cons = model_consumption_test(x_test)

print("   ✓ Forward pass successful")
for key, value in output_cons.items():
    print(f"   ✓ {key}: {value.shape}")

# Count parameters
total_params = sum(p.numel() for p in model_consumption_test.parameters())
print(f"   ✓ Total parameters: {total_params:,}")
print(f"   ✓ Model size: ~{total_params * 4 / (1024**2):.1f} MB (float32)")

In [ ]:
# Test 4: Consumption Loss function
print("\n4. Testing Consumption Loss function...")

criterion_cons = ConsumptionLoss(horizon_weights={'day': 1.0, 'week': 0.5})

targets = {
    'consumption_day': torch.FloatTensor(y_cons['consumption_day'][:2]),
    'consumption_week': torch.FloatTensor(y_cons['consumption_week'][:2])
}

loss, loss_dict = criterion_cons(output_cons, targets)
print("   ✓ Loss calculation successful")
for key, value in loss_dict.items():
    print(f"   ✓ {key}: {value:.4f}")

In [ ]:
# Test 5: Gradient flow
print("\n5. Testing gradient flow (Consumption Transformer)...")

model_consumption_test.train()
optimizer = torch.optim.AdamW(model_consumption_test.parameters(), lr=1e-4)

optimizer.zero_grad()
output = model_consumption_test(x_test)
loss, _ = criterion_cons(output, targets)
loss.backward()
optimizer.step()

print("   ✓ Gradient flow successful")
print("   ✓ Consumption Transformer can be trained")

In [ ]:
# Test 6: Trading Feature Engineering
print("\n6. Testing Trading Feature Engineering...")

engineer_trading = TradingFeatureEngineer()

# Get consumption predictions from test model
with torch.no_grad():
    test_predictions = model_consumption_test(torch.FloatTensor(X_cons[:100]))
    consumption_preds = test_predictions['consumption_day'].numpy()

# Prepare trading features
features_trading = engineer_trading.prepare_features(
    consumption_preds,
    df_complete.iloc[:len(consumption_preds)]
)

print(f"   ✓ Trading features extracted: {features_trading.shape}")
print(f"   ✓ Expected: (n_samples, 30)")
assert features_trading.shape[1] == 30, f"❌ Expected 30 features, got {features_trading.shape[1]}"

# Check for NaN/Inf
assert not np.isnan(features_trading).any(), "❌ NaN values in trading features!"
assert not np.isinf(features_trading).any(), "❌ Inf values in trading features!"
print("   ✓ No NaN/Inf values in trading features")

In [ ]:
# Test 7: Trading Transformer forward pass
print("\n7. Testing Trading Transformer forward pass (CPU)...")

model_trading_test = TradingTransformer(
    n_features=30,
    d_model=512,
    n_heads=8,
    n_layers=6,
    prediction_horizon=48
)

# Create sequences for transformer input (batch, seq_len=48, features=30)
# We need at least 48 samples to create a sequence
if len(features_trading) >= 48:
    # Take a sliding window of 48 timesteps
    x_trading_test = np.array([
        features_trading[0:48],   # First sequence
        features_trading[1:49]    # Second sequence (shifted by 1)
    ])
    x_trading_test = torch.FloatTensor(x_trading_test)
    print(f"   Created test sequences: {x_trading_test.shape}")
else:
    raise ValueError(f"Need at least 48 samples for trading test, got {len(features_trading)}")


with torch.no_grad():
    output_trading = model_trading_test(x_trading_test)

print("   ✓ Forward pass successful")
for key, value in output_trading.items():
    print(f"   ✓ {key}: {value.shape}")

# Count parameters
total_params_trading = sum(p.numel() for p in model_trading_test.parameters())
print(f"   ✓ Total parameters: {total_params_trading:,}")
print(f"   ✓ Model size: ~{total_params_trading * 4 / (1024**2):.1f} MB (float32)")

In [ ]:
# Test 8: Quick training test (2 epochs on small dataset)
print("\n8. Quick training test (2 epochs on CPU with small dataset)...")
print("   (This verifies the entire training pipeline works)")

# Create small data loaders
train_loader_test, val_loader_test, y_keys_test = create_data_loaders(
    X_cons[:100],
    {k: v[:100] for k, v in y_cons.items()},
    batch_size=8,
    train_split=0.8
)

model_consumption_test.train()
optimizer_test = torch.optim.AdamW(model_consumption_test.parameters(), lr=1e-4)

for epoch in range(2):
    train_loss, _ = train_epoch(
        model_consumption_test,
        train_loader_test,
        criterion_cons,
        optimizer_test,
        torch.device('cpu'),
        y_keys_test
    )
    val_loss, _ = validate(
        model_consumption_test,
        val_loader_test,
        criterion_cons,
        torch.device('cpu'),
        y_keys_test
    )
    print(f"   Epoch {epoch+1}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}")

print("   ✓ Training pipeline works correctly")

In [ ]:
# Clean up test models to free memory
print("\n9. Cleaning up test models...")
del model_consumption_test, model_trading_test
del x_test, x_trading_test
del train_loader_test, val_loader_test
import gc
gc.collect()
print("   ✓ Memory cleaned")

print("\n" + "="*70)
print("✅ PREFLIGHT VALIDATION PASSED")
print("All systems ready for GPU training!")
print("="*70)

## Section 4: Train Consumption Transformer (GPU)

Train the consumption prediction model on full dataset using GPU.

In [ ]:
print("="*70)
print("SECTION 4: TRAINING CONSUMPTION TRANSFORMER")
print("="*70)
print(f"\nTraining on device: {device}")
print(f"Dataset size: {len(X_cons):,} sequences\n")

In [ ]:
# Initialize model and move to GPU
print("\n1. Initializing Consumption Transformer...")

model_consumption = ConsumptionTransformer(
    n_features=17,
    d_model=384,
    n_heads=6,
    n_layers=5,
    dim_feedforward=1536,
    dropout=0.1,
    horizons={'day': 48, 'week': 336}
).to(device)

total_params = sum(p.numel() for p in model_consumption.parameters())
print(f"   ✓ Model created with {total_params:,} parameters")
print(f"   ✓ Model size: ~{total_params * 4 / (1024**2):.1f} MB")
print(f"   ✓ Moved to {device}")

In [ ]:
# Create data loaders
print("\n2. Creating data loaders...")

train_loader, val_loader, y_keys = create_data_loaders(
    X_cons,
    y_cons,
    batch_size=32,
    train_split=0.8
)

print(f"   ✓ Data loaders ready")
print(f"   ✓ Training batches: {len(train_loader)}")
print(f"   ✓ Validation batches: {len(val_loader)}")

In [ ]:
# Setup training
print("\n3. Setting up training...")

criterion = ConsumptionLoss(horizon_weights={'day': 1.0, 'week': 0.5})
optimizer = torch.optim.AdamW(
    model_consumption.parameters(),
    lr=1e-4,
    weight_decay=0.01
)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5,
    verbose=True
)

print("   ✓ Loss function: ConsumptionLoss (MSE weighted by horizon)")
print("   ✓ Optimizer: AdamW (lr=1e-4, weight_decay=0.01)")
print("   ✓ Scheduler: ReduceLROnPlateau (patience=5)")

In [ ]:
# Training loop
print("\n4. Starting training loop...")
print("   Target: MAPE < 15% on day-ahead predictions")
print("   Early stopping: patience = 10 epochs\n")

from tqdm.notebook import tqdm

num_epochs = 100
best_val_loss = float('inf')
patience = 10
patience_counter = 0

history = {
    'train_loss': [],
    'val_loss': [],
    'train_mape': [],
    'val_mape': []
}

for epoch in range(1, num_epochs + 1):
    print(f"\nEpoch {epoch}/{num_epochs}")
    print("-" * 50)
    
    # Train
    train_loss, train_metrics = train_epoch(
        model_consumption,
        train_loader,
        criterion,
        optimizer,
        device,
        y_keys
    )
    
    # Validate
    val_loss, val_metrics = validate(
        model_consumption,
        val_loader,
        criterion,
        device,
        y_keys
    )
    
    # Calculate MAPE on a small sample
    model_consumption.eval()
    with torch.no_grad():
        sample_x = torch.FloatTensor(X_cons[-100:]).to(device)
        sample_pred = model_consumption(sample_x)['consumption_day'].cpu().numpy()
        sample_target = y_cons['consumption_day'][-100:]
        val_mape = calculate_mape(sample_pred, sample_target)
    
    # Update scheduler
    scheduler.step(val_loss)
    
    # Track history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_mape'].append(val_mape)
    
    # Print metrics
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss:   {val_loss:.4f}")
    print(f"Val MAPE:   {val_mape:.2f}%")
    print(f"LR:         {optimizer.param_groups[0]['lr']:.6f}")
    
    # Save checkpoint if best
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        
        save_checkpoint(
            model_consumption,
            optimizer,
            epoch,
            {'train_loss': train_loss, 'val_loss': val_loss, 'val_mape': val_mape},
            '/content/drive/MyDrive/energymvp/checkpoints/consumption_transformer_best.pt'
        )
        print(f"✓ Best model saved (val_loss: {val_loss:.4f}, MAPE: {val_mape:.2f}%)")
    else:
        patience_counter += 1
        print(f"Patience: {patience_counter}/{patience}")
        
        if patience_counter >= patience:
            print(f"\nEarly stopping after {epoch} epochs")
            break
    
    # Save periodic checkpoints every 5 epochs
    if epoch % 5 == 0:
        save_checkpoint(
            model_consumption,
            optimizer,
            epoch,
            {'train_loss': train_loss, 'val_loss': val_loss},
            f'/content/drive/MyDrive/energymvp/checkpoints/consumption_epoch_{epoch}.pt'
        )

print("\n" + "="*70)
print("✅ CONSUMPTION TRANSFORMER TRAINING COMPLETE")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Final MAPE: {val_mape:.2f}%")
print("="*70)

In [ ]:
# Plot training history
print("\n5. Plotting training history...")

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].plot(history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Consumption Transformer - Training Loss')
axes[0].legend()
axes[0].grid(True)

# MAPE plot
axes[1].plot(history['val_mape'], label='Val MAPE', color='orange')
axes[1].axhline(y=15, color='r', linestyle='--', label='Target (15%)')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAPE (%)')
axes[1].set_title('Consumption Transformer - Validation MAPE')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/energymvp/checkpoints/consumption_training_history.png', dpi=150)
plt.show()

print("   ✓ Training history saved")

## Section 5: Train Trading Transformer (GPU)

Train the trading decision model using consumption predictions as input.

In [ ]:
print("="*70)
print("SECTION 5: TRAINING TRADING TRANSFORMER")
print("="*70)
print("\nThis model learns to make profitable trading decisions.\n")

In [ ]:
# Load best consumption model
print("\n1. Loading best Consumption Transformer...")

load_checkpoint(
    '/content/drive/MyDrive/energymvp/checkpoints/consumption_transformer_best.pt',
    model_consumption
)
model_consumption.eval()

print("   ✓ Best consumption model loaded")

In [ ]:
# Generate consumption predictions for all data
print("\n2. Generating consumption predictions for training data...")
print("   (This may take a few minutes)\n")

all_predictions = []
batch_size = 32

with torch.no_grad():
    for i in tqdm(range(0, len(X_cons), batch_size)):
        batch = torch.FloatTensor(X_cons[i:i+batch_size]).to(device)
        pred = model_consumption(batch)
        all_predictions.append(pred['consumption_day'].cpu().numpy())

consumption_predictions = np.vstack(all_predictions)
print(f"\n   ✓ Generated {len(consumption_predictions):,} predictions")
print(f"   ✓ Shape: {consumption_predictions.shape}")

In [ ]:
# Calculate optimal trading labels using V2 optimizer
print("\n3. Calculating optimal trading labels...")
print("   (Applying business rules to create training targets)\n")

optimal_decisions = []
optimal_quantities = []
optimal_prices = []

# Initialize battery at 35% SoC (room to buy)
initial_soc = 0.35
current_battery_charge = 40.0 * initial_soc

for i in tqdm(range(len(consumption_predictions))):
    # Battery state - track it ourselves with proper evolution
    battery_state = {
        'current_charge_kwh': current_battery_charge,
        'capacity_kwh': 40.0,
        'min_soc': 0.20,
        'max_soc': 1.0,  # Changed from 0.90
        'max_charge_rate_kw': 10.0,
        'max_discharge_rate_kw': 8.0,
        'efficiency': 0.95
    }
    
    # Get pricing data for next 48 intervals
    price_data = pricing_df['price_per_kwh'].values[i:i+48] if i+48 <= len(pricing_df) else pricing_df['price_per_kwh'].values[-48:]
    
    # Calculate optimal trading decision using V2 optimizer
    labels = calculate_optimal_trading_decisions(
        predicted_consumption=consumption_predictions[i],
        actual_prices=price_data,
        battery_state=battery_state,
        household_price_kwh=0.27,
        buy_threshold_mwh=20.0,
        sell_threshold_mwh=40.0,
        min_soc_for_sell=0.25,
        target_soc_on_buy=0.90
    )
    
    optimal_decisions.append(labels['optimal_decisions'])
    optimal_quantities.append(labels['optimal_quantities'])
    optimal_prices.append(price_data[0])
    
    # Update battery charge based on first decision
    decision = labels['optimal_decisions'][0]
    quantity = labels['optimal_quantities'][0]
    
    if decision == 0:  # Buy
        current_battery_charge += quantity * battery_state['efficiency']
    elif decision == 2:  # Sell
        current_battery_charge -= quantity
    
    # CRITICAL: Subtract consumption
    current_battery_charge -= consumption_predictions[i][0]
    
    # Clip to bounds
    current_soc = current_battery_charge / 40.0
    current_soc = np.clip(current_soc, 0.20, 1.0)
    current_battery_charge = 40.0 * current_soc

optimal_decisions = np.array(optimal_decisions)
optimal_quantities = np.array(optimal_quantities)
optimal_prices = np.array(optimal_prices)

print(f"\n   ✓ Calculated {len(optimal_decisions):,} optimal trading labels")
print(f"   ✓ Decision distribution:")
unique, counts = np.unique(optimal_decisions[:, 0], return_counts=True)
for decision, count in zip(unique, counts):
    decision_name = ['Buy', 'Hold', 'Sell'][int(decision)]
    print(f"      - {decision_name}: {count} ({count/len(optimal_decisions)*100:.1f}%)")

print(f"   ✓ Final battery SoC: {current_soc*100:.1f}%")

In [ ]:
# Prepare trading features
print("\n4. Preparing trading features...")

features_trading = engineer_trading.prepare_features(
    consumption_predictions,
    df_complete.iloc[:len(consumption_predictions)]
)

print(f"   ✓ Trading features prepared: {features_trading.shape}")
print(f"   ✓ Expected: (n_samples, 30)")

# Reshape for transformer (batch, seq_len=48, features=30)
# For now, we'll use a sliding window approach
# Each sample uses the last 48 feature vectors
n_samples_trading = len(features_trading) - 48
X_trading = np.zeros((n_samples_trading, 48, 30))

for i in range(n_samples_trading):
    X_trading[i] = features_trading[i:i+48]

# Align targets
y_trading = {
    'price': optimal_prices[:n_samples_trading],
    'decisions': optimal_decisions[:n_samples_trading, 0],  # Next interval decision
    'quantities': optimal_quantities[:n_samples_trading, 0],  # Next interval quantity
    'consumption': consumption_predictions[:n_samples_trading, 0]  # For profit calculation
}

print(f"   ✓ Trading sequences created: {X_trading.shape}")
print(f"   ✓ Target shapes:")
for key, value in y_trading.items():
    print(f"      - {key}: {value.shape}")

In [ ]:
# Initialize Trading Transformer
print("\n5. Initializing Trading Transformer...")

model_trading = TradingTransformer(
    n_features=30,
    d_model=512,
    n_heads=8,
    n_layers=6,
    dim_feedforward=2048,
    dropout=0.1,
    prediction_horizon=48
).to(device)

total_params_trading = sum(p.numel() for p in model_trading.parameters())
print(f"   ✓ Model created with {total_params_trading:,} parameters")
print(f"   ✓ Model size: ~{total_params_trading * 4 / (1024**2):.1f} MB")
print(f"   ✓ Moved to {device}")

In [ ]:
# Create data loaders for trading
print("\n6. Creating data loaders for trading...")

train_loader_trading, val_loader_trading, y_keys_trading = create_data_loaders(
    X_trading,
    y_trading,
    batch_size=32,
    train_split=0.8
)

print(f"   ✓ Data loaders ready")

In [ ]:
# Setup training for Trading Transformer
print("\n7. Setting up trading training...")

criterion_trading = TradingLoss(
    price_weight=0.20,
    decision_weight=0.20,
    profit_weight=0.60,
    household_price_kwh=0.27
)

optimizer_trading = torch.optim.AdamW(
    model_trading.parameters(),
    lr=1e-4,
    weight_decay=0.01
)

scheduler_trading = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_trading,
    mode='min',
    factor=0.5,
    patience=5,
    verbose=True
)

print("   ✓ Loss: TradingLoss (20% price + 20% decision + 60% profit)")
print("   ✓ Optimizer: AdamW (lr=1e-4)")
print("   ✓ Scheduler: ReduceLROnPlateau")

In [ ]:
# Trading Transformer training loop
print("\n8. Starting trading training loop...")
print("   Target: Price MAE < $0.05/kWh, Cost savings 20-40%")
print("   Early stopping: patience = 10 epochs\n")

num_epochs_trading = 100
best_val_loss_trading = float('inf')
patience_trading = 10
patience_counter_trading = 0

history_trading = {
    'train_loss': [],
    'val_loss': [],
    'price_mae': [],
    'avg_profit': []
}

for epoch in range(1, num_epochs_trading + 1):
    print(f"\nEpoch {epoch}/{num_epochs_trading}")
    print("-" * 50)
    
    # Train
    train_loss, train_metrics = train_epoch(
        model_trading,
        train_loader_trading,
        criterion_trading,
        optimizer_trading,
        device,
        y_keys_trading
    )
    
    # Validate
    val_loss, val_metrics = validate(
        model_trading,
        val_loader_trading,
        criterion_trading,
        device,
        y_keys_trading
    )
    
    # Update scheduler
    scheduler_trading.step(val_loss)
    
    # Track history
    history_trading['train_loss'].append(train_loss)
    history_trading['val_loss'].append(val_loss)
    
    # Print metrics
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss:   {val_loss:.4f}")
    print(f"LR:         {optimizer_trading.param_groups[0]['lr']:.6f}")
    
    # Save checkpoint if best
    if val_loss < best_val_loss_trading:
        best_val_loss_trading = val_loss
        patience_counter_trading = 0
        
        save_checkpoint(
            model_trading,
            optimizer_trading,
            epoch,
            {'train_loss': train_loss, 'val_loss': val_loss},
            '/content/drive/MyDrive/energymvp/checkpoints/trading_transformer_best.pt'
        )
        print(f"✓ Best trading model saved (val_loss: {val_loss:.4f})")
    else:
        patience_counter_trading += 1
        print(f"Patience: {patience_counter_trading}/{patience_trading}")
        
        if patience_counter_trading >= patience_trading:
            print(f"\nEarly stopping after {epoch} epochs")
            break

print("\n" + "="*70)
print("✅ TRADING TRANSFORMER TRAINING COMPLETE")
print(f"Best validation loss: {best_val_loss_trading:.4f}")
print("="*70)

In [ ]:
# Plot trading training history
print("\n9. Plotting trading training history...")

fig, ax = plt.subplots(1, 1, figsize=(10, 5))

ax.plot(history_trading['train_loss'], label='Train Loss')
ax.plot(history_trading['val_loss'], label='Val Loss')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Trading Transformer - Training Loss')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/energymvp/checkpoints/trading_training_history.png', dpi=150)
plt.show()

print("   ✓ Training history saved")

## Section 6: End-to-End Testing

Validate the complete pipeline with comprehensive metrics.

In [ ]:
print("="*70)
print("SECTION 6: END-TO-END TESTING")
print("="*70)
print("\nRunning complete inference pipeline on test data...\n")

In [ ]:
# Load best models
print("\n1. Loading best models...")

load_checkpoint(
    '/content/drive/MyDrive/energymvp/checkpoints/consumption_transformer_best.pt',
    model_consumption
)
load_checkpoint(
    '/content/drive/MyDrive/energymvp/checkpoints/trading_transformer_best.pt',
    model_trading
)

model_consumption.eval()
model_trading.eval()

print("   ✓ Both models loaded and set to eval mode")

In [ ]:
# Run inference on test set
print("\n2. Running inference on test samples...")

test_size = 100
test_X_cons = torch.FloatTensor(X_cons[-test_size:]).to(device)
test_X_trading = torch.FloatTensor(X_trading[-test_size:]).to(device)

with torch.no_grad():
    # Consumption predictions
    consumption_pred = model_consumption(test_X_cons)
    
    # Trading predictions
    trading_pred = model_trading(test_X_trading)

print("   ✓ Inference complete")

In [ ]:
# Calculate comprehensive metrics
print("\n3. Calculating metrics...")

# Consumption MAPE
cons_pred_day = consumption_pred['consumption_day'].cpu().numpy()
cons_target_day = y_cons['consumption_day'][-test_size:]
consumption_mape = calculate_mape(cons_pred_day, cons_target_day)

# Price MAE
price_pred = trading_pred['predicted_price'][:, 0].cpu().numpy()
price_target = y_trading['price'][-test_size:]
price_mae = np.mean(np.abs(price_pred - price_target))

# Trading decision accuracy
decision_pred = torch.argmax(trading_pred['trading_decisions'][:, 0, :], dim=1).cpu().numpy()
decision_target = y_trading['decisions'][-test_size:]
decision_accuracy = (decision_pred == decision_target).mean() * 100

print(f"\n   📊 PERFORMANCE METRICS:")
print(f"   {'='*50}")
print(f"   Consumption MAPE:      {consumption_mape:.2f}% {'✓' if consumption_mape < 15 else '❌'} (target < 15%)")
print(f"   Price MAE:             ${price_mae:.4f} {'✓' if price_mae < 0.05 else '❌'} (target < $0.05)")
print(f"   Trading Accuracy:      {decision_accuracy:.2f}%")
print(f"   {'='*50}")

In [ ]:
# Calculate cost savings vs baseline
print("\n4. Calculating cost savings vs baseline...")

# Baseline: buy all energy from grid at market price
baseline_cost = (cons_target_day * price_target[:, np.newaxis]).sum()

# Optimized: use trading decisions
# This is a simplified calculation - full simulation would be more complex
quantities_pred = trading_pred['trade_quantities'][:, 0].cpu().numpy()
buy_mask = (decision_pred == 0)
sell_mask = (decision_pred == 2)

trading_cost = baseline_cost  # Start with baseline
trading_cost -= (quantities_pred[sell_mask] * price_target[sell_mask]).sum()  # Revenue from selling
trading_cost += (quantities_pred[buy_mask] * price_target[buy_mask]).sum()  # Cost of buying

savings = baseline_cost - trading_cost
savings_percent = (savings / baseline_cost) * 100

print(f"\n   💰 COST ANALYSIS:")
print(f"   {'='*50}")
print(f"   Baseline Cost:         ${baseline_cost:.2f}")
print(f"   Optimized Cost:        ${trading_cost:.2f}")
print(f"   Savings:               ${savings:.2f}")
print(f"   Savings Percentage:    {savings_percent:.2f}% {'✓' if 20 <= savings_percent <= 40 else '⚠️'} (target 20-40%)")
print(f"   {'='*50}")

In [ ]:
# Trading decision confusion matrix
print("\n5. Trading decision distribution...")

from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(decision_target, decision_pred)
labels = ['Buy', 'Hold', 'Sell']

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Trading Decision Confusion Matrix')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/energymvp/checkpoints/confusion_matrix.png', dpi=150)
plt.show()

print("   ✓ Confusion matrix saved")

In [ ]:
# Visualize sample predictions
print("\n6. Visualizing sample predictions...")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Consumption prediction
sample_idx = 0
axes[0, 0].plot(cons_target_day[sample_idx], label='Actual', marker='o', markersize=3)
axes[0, 0].plot(cons_pred_day[sample_idx], label='Predicted', marker='x', markersize=3)
axes[0, 0].set_xlabel('Time Interval')
axes[0, 0].set_ylabel('Consumption (kWh)')
axes[0, 0].set_title('Consumption Prediction (Next 24 Hours)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Price prediction
axes[0, 1].scatter(price_target, price_pred, alpha=0.5)
axes[0, 1].plot([price_target.min(), price_target.max()], 
                [price_target.min(), price_target.max()], 
                'r--', label='Perfect Prediction')
axes[0, 1].set_xlabel('Actual Price ($/kWh)')
axes[0, 1].set_ylabel('Predicted Price ($/kWh)')
axes[0, 1].set_title('Price Prediction Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Trading decisions over time
axes[1, 0].plot(decision_pred[:50], marker='o', label='Predicted')
axes[1, 0].plot(decision_target[:50], marker='x', label='Optimal', alpha=0.7)
axes[1, 0].set_xlabel('Time Interval')
axes[1, 0].set_ylabel('Decision (0=Buy, 1=Hold, 2=Sell)')
axes[1, 0].set_title('Trading Decisions (First 50 Intervals)')
axes[1, 0].set_yticks([0, 1, 2])
axes[1, 0].set_yticklabels(['Buy', 'Hold', 'Sell'])
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Training history comparison
axes[1, 1].plot(history['val_loss'], label='Consumption Val Loss')
axes[1, 1].plot(history_trading['val_loss'], label='Trading Val Loss')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].set_title('Training Convergence')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_yscale('log')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/energymvp/checkpoints/end_to_end_results.png', dpi=150)
plt.show()

print("   ✓ Visualizations saved")

print("\n" + "="*70)
print("✅ END-TO-END TESTING COMPLETE")
print("="*70)

## Section 7: Model Export & Summary

Save model configurations, training logs, and generate final summary report.

In [ ]:
print("="*70)
print("SECTION 7: MODEL EXPORT & SUMMARY")
print("="*70)

In [ ]:
# Save model configurations
print("\n1. Saving model configurations...")

import json

config_consumption = {
    'architecture': 'ConsumptionTransformer',
    'n_features': 17,
    'd_model': 384,
    'n_heads': 6,
    'n_layers': 5,
    'dim_feedforward': 1536,
    'horizons': {'day': 48, 'week': 336},
    'total_parameters': sum(p.numel() for p in model_consumption.parameters())
}

config_trading = {
    'architecture': 'TradingTransformer',
    'n_features': 30,
    'd_model': 512,
    'n_heads': 8,
    'n_layers': 6,
    'dim_feedforward': 2048,
    'prediction_horizon': 48,
    'total_parameters': sum(p.numel() for p in model_trading.parameters())
}

with open('/content/drive/MyDrive/energymvp/checkpoints/consumption_config.json', 'w') as f:
    json.dump(config_consumption, f, indent=2)

with open('/content/drive/MyDrive/energymvp/checkpoints/trading_config.json', 'w') as f:
    json.dump(config_trading, f, indent=2)

print("   ✓ Model configurations saved")

In [ ]:
# Save training logs
print("\n2. Saving training logs...")

training_logs = {
    'consumption': {
        'train_loss_history': [float(x) for x in history['train_loss']],
        'val_loss_history': [float(x) for x in history['val_loss']],
        'val_mape_history': [float(x) for x in history['val_mape']],
        'best_val_loss': float(best_val_loss),
        'final_mape': float(consumption_mape),
        'epochs_trained': len(history['train_loss'])
    },
    'trading': {
        'train_loss_history': [float(x) for x in history_trading['train_loss']],
        'val_loss_history': [float(x) for x in history_trading['val_loss']],
        'best_val_loss': float(best_val_loss_trading),
        'epochs_trained': len(history_trading['train_loss'])
    },
    'evaluation': {
        'consumption_mape': float(consumption_mape),
        'price_mae': float(price_mae),
        'trading_accuracy': float(decision_accuracy),
        'baseline_cost': float(baseline_cost),
        'optimized_cost': float(trading_cost),
        'savings': float(savings),
        'savings_percent': float(savings_percent)
    },
    'training_date': datetime.now().isoformat(),
    'device': str(device)
}

with open('/content/drive/MyDrive/energymvp/checkpoints/training_logs.json', 'w') as f:
    json.dump(training_logs, f, indent=2)

print("   ✓ Training logs saved")

In [ ]:
# Generate final summary report
print("\n3. Generating summary report...")

summary = f"""
{'='*70}
DUAL-TRANSFORMER ENERGY TRADING SYSTEM
Training Summary Report
{'='*70}

Training Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Device: {device}

{'='*70}
CONSUMPTION TRANSFORMER
{'='*70}
Architecture:        {config_consumption['d_model']}d, {config_consumption['n_heads']} heads, {config_consumption['n_layers']} layers
Parameters:          {config_consumption['total_parameters']:,}
Input Features:      {config_consumption['n_features']}
Output Horizons:     Day (48), Week (336)

Training:
  Epochs:            {len(history['train_loss'])}
  Best Val Loss:     {best_val_loss:.4f}
  Final MAPE:        {consumption_mape:.2f}%
  Target:            < 15% {'✓ PASSED' if consumption_mape < 15 else '❌ NOT MET'}

{'='*70}
TRADING TRANSFORMER
{'='*70}
Architecture:        {config_trading['d_model']}d, {config_trading['n_heads']} heads, {config_trading['n_layers']} layers
Parameters:          {config_trading['total_parameters']:,}
Input Features:      {config_trading['n_features']}
Output:              Price + Decision + Quantity

Training:
  Epochs:            {len(history_trading['train_loss'])}
  Best Val Loss:     {best_val_loss_trading:.4f}

{'='*70}
EVALUATION METRICS
{'='*70}
Consumption MAPE:    {consumption_mape:.2f}% {'✓' if consumption_mape < 15 else '❌'} (target < 15%)
Price MAE:           ${price_mae:.4f} {'✓' if price_mae < 0.05 else '❌'} (target < $0.05)
Trading Accuracy:    {decision_accuracy:.2f}%

Cost Analysis:
  Baseline Cost:     ${baseline_cost:.2f}
  Optimized Cost:    ${trading_cost:.2f}
  Savings:           ${savings:.2f}
  Savings %:         {savings_percent:.2f}% {'✓' if 20 <= savings_percent <= 40 else '⚠️'} (target 20-40%)

{'='*70}
SUCCESS CRITERIA
{'='*70}
✓ Consumption MAPE < 15%:     {'✓ PASSED' if consumption_mape < 15 else '❌ NOT MET'}
✓ Price MAE < $0.05/kWh:      {'✓ PASSED' if price_mae < 0.05 else '❌ NOT MET'}
✓ Cost savings 20-40%:        {'✓ PASSED' if 20 <= savings_percent <= 40 else '⚠️ NOT MET'}
✓ Models converged:           ✓ PASSED
✓ Checkpoints saved:          ✓ PASSED

{'='*70}
FILES SAVED
{'='*70}
Checkpoints:
  - consumption_transformer_best.pt
  - trading_transformer_best.pt

Configurations:
  - consumption_config.json
  - trading_config.json

Logs:
  - training_logs.json

Visualizations:
  - consumption_training_history.png
  - trading_training_history.png
  - confusion_matrix.png
  - end_to_end_results.png

{'='*70}
NEXT STEPS
{'='*70}
1. Download checkpoints from Google Drive
2. Test models on local machine
3. Deploy to production environment
4. Monitor performance metrics
5. Retrain periodically with new data

{'='*70}
TRAINING COMPLETE
{'='*70}
"""

print(summary)

# Save summary to file
with open('/content/drive/MyDrive/energymvp/checkpoints/TRAINING_SUMMARY.txt', 'w') as f:
    f.write(summary)

print("\n   ✓ Summary report saved to TRAINING_SUMMARY.txt")

In [ ]:
# Final memory cleanup
print("\n4. Final cleanup...")

if torch.cuda.is_available():
    print(f"   GPU memory used: {torch.cuda.memory_allocated(0) / 1024**2:.1f} MB")
    torch.cuda.empty_cache()
    print(f"   GPU memory after cleanup: {torch.cuda.memory_allocated(0) / 1024**2:.1f} MB")

print("\n" + "="*70)
print("✅ ALL TRAINING COMPLETE")
print("="*70)
print("\nAll models, logs, and visualizations saved to:")
print("/content/drive/MyDrive/energymvp/checkpoints/")
print("\nReady for deployment!")
print("="*70)